# Image Processing End of Term Project
# Gate Access Controller Using License Plate Detection and OCR
# Team 14


In [1]:
! pip install opencv-python==4.5.4.60

In [2]:
###Imports

# Preprocessing and chaarcter recognition modules
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from skimage.color import rgb2gray,rgb2hsv
from scipy.ndimage import gaussian_filter
from scipy import signal as sig
from scipy.signal import convolve2d
from scipy import fftpack
import math
from skimage.util import random_noise
from skimage.filters import median
from skimage.feature import canny
from skimage.measure import label
from skimage.filters import threshold_local
from skimage.color import label2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import skeletonize
import imutils
from skimage import measure
from PIL import Image
import pytesseract
import easyocr
import PySimpleGUI as sg
from PIL import Image, ImageTk
#import pkg_resources
#pkg_resources.require('OpenCV==4.5.4.60')
import cv2
print('OpenCV version: ', cv2.__version__)

OpenCV version:  4.5.4


In [3]:
#Function to display images
def show_images(images,titles=None):
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 


# Preprocessing step

#### 1. First we resize image for preprocessing to be same for all images
#### 2. We Convert to gray scale and using bilateral filter to eliminate noise, as it is more effectve than median filter 
#### 3. Next we use Canny edge detector to extract edges because has good localization, it extract image features without altering the image features and it is less sensitive to noise
#### 4. Create a mask for the contour only

In [4]:
# Different noise eliminating filters

def MedianFilter(image):
    return median(img)

def BilateralFilter(image):
    return cv2.bilateralFilter(image, 11, 17, 17)

In [5]:
def preprocessing(image):
    #Handling corner case of image not exists
    if image is None: 
        print('This image does not exist.')
        return None, None, 0, 0, 0, 0, None
    
    cv2.resize(image,(int(image.shape[0]),int(500)))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_filtered = cv2.bilateralFilter(gray, 11, 17, 17)
    edged=cv2.Canny(gray_filtered,170,200)
    
    # Finding enclosed areas which is the license plate  we get contours of minimum size 10
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # We store te location of the largest contour found in our image
    # we know it's the largest since we sort contours using the contour area
    location = None
    # No contour of minimum size 10 found
    if len(contours) == 0: return None, None, 0, 0, 0, 0, None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        # Retrieve contours with 4 vertices, therefore rectangular, if found break
        if len(approx) == 4:
            location = approx
            x,y,w,h = cv2.boundingRect(contour)
            break
    
    # Contour location might not be found so handle if none
    if location is None: return None, None, 0, 0, 0, 0, None
    
    # We create a black image of all zeros to draw contour found on 
    mask = np.zeros(gray_filtered.shape, np.uint8)
    drawCont = cv2.drawContours(mask, [location], 0,255, -1)
    Masked = cv2.bitwise_and(image, image, mask = mask)
    
   
    # Returns the plate region processed, the x,y coordinates of the plate, the width and heigh of the region
    # and finally the masked image before ccropping into plateRegionOnly
    #----------
    mask = np.zeros(gray.shape, np.uint8)
    if location is None: return None, None, 0, 0, 0, 0, None
    new_image1 = cv2.drawContours(mask, [location], 0,255, -1)
    new_image2 = cv2.bitwise_and(image, image, mask=mask)
    
    ############################################ Step 2 -> Plate Enhancement
    #SE_Size=100
    kernel =cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

    #extractedPlate = cv2.erode(new_image2,kernel,iterations = 1)
    #img_erosion = cv2.erode(img, kernel, iterations=1)
    #img_dilation = cv2.dilate(extractedPlate, kernel, iterations=2)
    
    plateRegionOnly = new_image2[y:y+h, x:x+w ]
    gray2 = cv2.cvtColor(plateRegionOnly, cv2.COLOR_RGB2GRAY)
    done = cv2.bilateralFilter(gray2, 11, 17, 17)
    #show_images([plateRegionOnly, Masked, new_image2, new_image1, done])

    return plateRegionOnly, location, x, y, w, h, Masked

#readImage = cv2.imread(pathAsStr+str(0)+'.png')
#plateRegionOnly, location, x, y, w, h, Masked = preprocessing(readImage)
#show_images([RegionOfInterest(plateRegionOnly, location, x, y, w, h)])

### This function is a customizable preprocessing function
### we can send it a list of 0s and 1's where 0 means erode and 1 means dilate and we pass the image through
### these as sent in order e.g. [0, 1, 1, 0] means an erossion, 2 dilations, and an erosion again

In [6]:
def customizablePreprocessing(image, seqOfErodDil, sizeStructElement):
    #Handling corner case of image not exists
    
    if image is None: 
        print('This image does not exist.')
        return None, None, 0, 0, 0, 0, 0
    
    cv2.resize(image,(int(image.shape[0]),int(500)))
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_filtered = cv2.bilateralFilter(gray, 11, 17, 17)
    edged=cv2.Canny(gray_filtered,170,200)
    
    # Finding enclosed areas which is the license plate  we get contours of minimum size 10
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # We store te location of the largest contour found in our image
    # we know it's the largest since we sort contours using the contour area
    location = None
    # No contour of minimum size 10 found
    if len(contours) == 0: return None, None, 0, 0, 0, 0, 0
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        # Retrieve contours with 4 vertices, therefore rectangular, if found break
        if len(approx) == 4:
            location = approx
            x,y,w,h = cv2.boundingRect(contour)
            break
    # Contour location might not be found so handle if none
    if location is None: return None, None, 0, 0, 0, 0 , 0
    
    # We create a black image of all zeros to draw contour found on 
    mask = np.zeros(gray.shape, np.uint8)
    drawCont = cv2.drawContours(mask, [location], 0,255, -1)
    Masked = cv2.bitwise_and(image, image, mask = mask)
    
    if seqOfErodDil is None: 
        return plateRegionOnly, location, x, y, w, h, Masked
    
    # else create our kernel
    kernel =cv2.getStructuringElement(cv2.MORPH_RECT,(sizeStructElement,sizeStructElement))
    lastImge = Masked
    for j in seqOfErodDil:
        if j == 0:
            morphology = cv2.erode(lastImge, kernel, iterations=1)
        else:
            morphology = cv2.dilate(extractedPlate, kernel, iterations=1)
        lastImge = morphology
    
    plateRegionOnly = lastImge[y:y+h, x:x+w ]
    return plateRegionOnly, location, x, y, w, h, lastImge

In [7]:
# Archived function because 
# 1. Causes error in some images 
# 2. Tilts are so slight and we assume camera position fitted to see plate in correct position
# 3. Even tilted iimages not affected by rotations

def RotateTilted(extractedPlate, location):
    left_bottom =  location[3][0]
    right_bottom = location[0][0]
    left_top =  location[2][0]
    right_top = location[1][0]
    xEnd = max(right_top[0],right_bottom[0]) + 20
    yEnd = min(right_bottom[1],left_bottom[1]) + 10
    plateRegionOnly = extractedPlate[yEnd:left_top[1]+10,left_bottom[0]-20:xEnd ]
    print(yEnd, left_top[1],left_bottom[0]-20,xEnd)
    hypotenuse =  math.pow((math.pow((left_bottom[0]-right_bottom[0]), 2) + math.pow((left_bottom[1]-right_bottom[1]), 2)), 0.5)
    opposite = right_bottom[1] - left_bottom[1]
    rotationAngle = math.asin(opposite/hypotenuse)*57.2958
    rotated = imutils.rotate_bound(plate, angle=-rotationAngle)
    return rotated


In [8]:
def RegionOfInterest(plate,location,x,y,w,h):    
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    # Get The Thresholding TODO:get another
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.medianBlur(gray, 3)
    GlobalThresh = threshold_otsu(gray)
    ThreshImage = np.copy(gray)
    ThreshImage[gray >= GlobalThresh] = 0
    ThreshImage[gray < GlobalThresh] = 1
    Parition = np.copy(ThreshImage)
    Binary = np.copy(plate)
    Parition = cv2.medianBlur(Parition, 3)
    return Parition

In [9]:
# Function segments characters by finding contours around character
def segmnetCharactersByContours(plate):
    copy=np.copy(plate)
    cnts = cv2.findContours(plate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    charsSegmented = []
    ROI_number = 0
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = plate[y:y+h, x:x+w]
        resizing = imutils.resize(ROI, width=200)
        charsSegmented.append(ROI)
        cv2.imwrite('result/ROI_{}.png'.format(ROI_number), 255*resizing)
        ROI_number += 1
    return charsSegmented

In [27]:
def segmnetCharactersByContours2(image):
    copy=np.copy(image)
    all_imgs=[] #to sort charchters as in the input image #imge,startingx,width
    cnts = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True) #discard first two detected imaged
    charsSegmented = []
    ROI_number = 0
    i=0
    #print(cnts)
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        outside = True
        if i < 2:
                ROI = image[y:y+h, x:x+w]
                resizing = imutils.resize(ROI, width=200)
                charsSegmented.append(ROI)
                cv2.imwrite('ROI_{}.jpg'.format(ROI_number), 255*resizing)
                cv2.rectangle(copy,(x,y),(x+w,y+h),(0,255,0),5)
                #show_images([image])
                ROI_number += 1 
        else: 
            for j in all_imgs:
                if x >  j[1] and x < j[1]+j[2]-1:
                    outside = False
            if outside:
                    ROI = image[y:y+h, x:x+w]
                    resizing = imutils.resize(ROI, width=200)
                    charsSegmented.append(ROI)
                    cv2.imwrite('ROI_{}.jpg'.format(ROI_number), 255*resizing)
                    cv2.rectangle(copy,(x,y),(x+w,y+h),(0,255,0),5)
                    #show_images([image])
                    ROI_number += 1
                    all_imgs.append((ROI,x,w))       
        i = i +  1
    #sort charchters as the same in the input image       
    all_imgs.sort(key=lambda start:start[1],reverse=False) #all_imgs[0]
    out_imgs=[]
    for char in all_imgs:
        out_imgs.append(char[0])
    #print(ROI_number)    
    return out_imgs #charsSegmented#

In [10]:
# Tested as better alternative to segmentation, is to directly use OCR
# Here we use improved processing ven further to get the characters and in order

# Sort characters by their order
def Sort_Tuple(listtup): 
    return listtup.sort(key=lambda y: y[0])

def connectedComponentSegmentation(threshAdjusted):
    # construct the argument parser and parse the arguments
    #ap = argparse.ArgumentParser()
    path = 'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/result/partition.png'
    output = cv2.connectedComponentsWithStats(threshAdjusted.astype(np.uint8), cv2.CV_32S)
    # Returns number of labels/compoennts, 
    (numLabels, labels, stats, centroids) = output
    #print('type is ', type(output))
    #print(numLabels,'\n', labels,'\n', stats,'\n', centroids)
    mask = np.zeros(threshAdjusted.shape, dtype="uint8")
    # Label 0 is the background
    ListOfTuples = []
    for i in range(1, numLabels):
        area = stats[i, cv2.CC_STAT_AREA]
        x = stats[i, cv2.CC_STAT_LEFT]
        labelTuple = (x, area, labels[i],stats[i],centroids[i])
        ListOfTuples.append(labelTuple)
        componentMask = (labels == i).astype("uint8") * 255
        mask = cv2.bitwise_or(mask, componentMask)
        mask = np.zeros(threshAdjusted.shape, dtype="uint8")
    # Now we have tuple of the character and added the x of them, we will sort on the x
    ListOfSortedTuples = []
    Sort_Tuple(ListOfTuples)
    for i in range(1, len(ListOfTuples)):
        componentMask = (labels == i).astype("uint8") * 255
        mask = cv2.bitwise_or(mask, componentMask)
        x, area, _,_,_ = ListOfTuples[i]
        #show_images([mask])
        mask = np.zeros(threshAdjusted.shape, dtype="uint8")
        ListOfSortedTuples.append(mask)
        
    return ListOfSortedTuples

In [11]:
reader = easyocr.Reader(['en']) # need to run only once to load model into memory

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [12]:
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def PytesseractRecognition(binary, colored):
    textUsingPytesBin = pytesseract.image_to_string(binary)
    textUsingPytesCol = pytesseract.image_to_string(colored)
    return textUsingPytesBin, textUsingPytesCol

def EasyOCR(binary, colored):
    textUsingBinary = reader.readtext(binary)
    textUsingColored = reader.readtext(colored)
    return textUsingBinary, textUsingColored

In [17]:
directory = r'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/result/'
pathAsStr = "C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/dataset/images/Cars"
def onTest(index):
    readImage = cv2.imread(pathAsStr+str(index)+'.png')
    extractedPlate, location, a, b, c, d, masked  = preprocessing(readImage)
    #show_images([extractedPlate,masked ])
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
    # We test on 4 different ways
    # Using pytesseract on the binary processed vs just cropped colored image 
    # and using easyOCR on the binary processed vs just cropped colored image  
    if (extractedPlate is not None and masked is not None):
        textUsingPytesBin, textUsingPytesCol = PytesseractRecognition(extractedPlate, masked)
        textUsingBinary, textUsingColored = EasyOCR(extractedPlate, masked)

        #print(textUsingPytesBin, '  using pytesseract on binary')
        #print(textUsingBinary, ' using easyOCR on binary')
        #print(textUsingColored, ' using easyOCR on colored')

    if location is not None:
        partition = RegionOfInterest(extractedPlate, location, a, b, c, d)
        loft = connectedComponentSegmentation(partition)
        #if loft is not None:
            #print(pytesseract.image_to_string(i) for i in loft)
    return textUsingColored
    
    
# 159

In [18]:
directory = r'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/result/'
pathAsStr = "C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/dataset/images/Cars"
index = 147
readImage = cv2.imread(pathAsStr+str(index)+'.png')
extractedPlate, location, a, b, c,d, colored  = preprocessing(readImage)
if location is not None:
    partition = RegionOfInterest(extractedPlate, location, a, b, c, d)
    ListOfSegChars = segmnetCharactersByContours(partition)
    #show_images(ListOfSegChars)
    filename = directory+'done'+str(index)+'.png'
    cv2.imwrite(filename,partition*255)

In [19]:
def acuraccy(ocr_otput,input_file): # ocr_output-> read from the ocr, input_file-> read from the input file
    plates=0 ## counter for whole plates acuraccy
    correct=0 
    uncorrect=0
    illuminate_space=ocr_otput.replace(" ","") # remove spaces
    capital_letter =  illuminate_space.upper() #change lower case to upper
    print(input_file,' vs ', capital_letter)
    if capital_letter==input_file:## whole matching
        plates=plates+1
        correct=len(capital_letter)
    else:
        for i in range(min(len(input_file), len(capital_letter))):
            if input_file[i]==capital_letter[i]:
                correct=correct+1
            else:
                uncorrect=uncorrect+1
    return plates,correct,uncorrect ## return: "plates->1 means whole matching, 0 means there's error","correct->number of correct char","uncorrect->number of uncorrect char"


In [26]:
def testAndCompare(filepathIndeces, filepathCorrect):
    my_file = open(filepathIndeces, "r")
    content = my_file.read()
    content_list = content.split(",")
    my_file.close()
    another = open(filepathCorrect, "r")
    contentChar = another.read()
    charPlate = contentChar.split(",")
    my_file.close()
    indeces = [int(num) for num in content_list]
    stringCorrect = []
    #print(indeces) # Just to check
    #print(charPlate)
    print(len(indeces) , len(charPlate))
    correctPlates = 0
    correctChars = 0 
    inCorrectChars = 0
    for i in range(len(indeces)):    
        x = onTest(indeces[i])
        yehemena = x[0][-2]
        plates, correct, uncorrect = acuraccy(yehemena,charPlate[i])
        correctPlates+=plates
        correctChars+=correct
        inCorrectChars+=uncorrect
    print('Number of cocorrect chars = ',correctChars )
    print('Number of incorrect chars = ',inCorrectChars )
    print('Number of completely correct plates = ', correctPlates)
    print('Accuracy is ', correctChars*100/(correctChars+inCorrectChars),'%. ' )
    
testAndCompare('TestSetImages.txt', 'carsNum(1).txt')

30 30
KLG1CA2555  vs  KLO1CA2555
PGMN112  vs  POE1
DZI7YXR  vs  DZI7YXR
WOR5167  vs  CRSI6K
ALR486  vs  AR4BB
DZI7YXR  vs  DZI7YXR
DL7CN5617  vs  @LZCN5617
MH15BD8877  vs  HH15BD8877
CHIOOSE  vs  CHIOOSE
TAXI  vs  TAXI
DL7CN5617  vs  @LZCN5617
TN99F2378  vs  TN9962378
CZI7KOD  vs  CZI7KOD
PGMN112  vs  POE1
KAG5MG19G9  vs  KA
YWORRY  vs  YPRRY
MK-35-32  vs  MK-35-32
KAG5MG19G9  vs  KA
KLG1CA2555  vs  KLO1CA2555
DL8CX4850  vs  DL8CX4850
LR33TEE  vs  LR33TEB
CZI7KOD  vs  CZI7KOD
KL65H4383  vs  KL65H4383
MH20EE7598  vs  MH20EE7598
EAB0001  vs  EABC
DZI7YXR  vs  DZI7YXR
MH20EJ0364  vs  NH20EJ0364
GOOGLE  vs  GOOGLE
GOOGLE  vs  GOOGLE
M771276  vs  M:
Number of cocorrect chars =  171
Number of incorrect chars =  31
Number of completely correct plates =  13
Accuracy is  84.65346534653466 %. 


In [ ]:
# This automates the preprocessing for the whole dataset
def automateWhole():
    directory = r'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/meow/'
    countNoRegion = 0
    for i in range(433):
        pathAsStr = "C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/dataset/images/Cars"
        readImage = cv2.imread(pathAsStr+str(i)+'.png')
        #show_images([readImage])
        extractedPlate, location, x, y, w, h = preprocessing(readImage)
        if location is not None:
            partition = RegionOfInterest(extractedPlate, location, x, y, w, h)
            #ListOfSegChars = segmnetCharactersByContours(partition)
            filename = directory+'done'+str(i)+'.png'
            cv2.imwrite(filename,partition*255)
            SortedChars = connectedComponentSegmentation(partition)
            if SortedChars is not None:
                for j in range(len(SortedChars)):
                    filename = directory+'done/'+str(i)+'7arf'+str(j)+'.png'
                    cv2.imwrite(filename,partition*255)
        else:
            countNoRegion+=1
    print(countNoRegion)
automateWhole()


In [ ]:
def automate():
    directory = r'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/result/'
    for i in range(433):
        pathAsStr = "C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/dataset/images/Cars"
        readImage = cv2.imread(pathAsStr+str(i)+'.png')
        #show_images([readImage])
        extractedPlate, location = preprocessing(readImage)
        if location is not None:
            partition = RegionOfInterest(extractedPlate, location)
            ListOfSegChars = segmnetCharactersByContours(partition)
            filename = directory+'done'+str(i)+'.png'
            cv2.imwrite(filename,partition*255)
automate()

In [ ]:
directory = r'C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/result/'
pathAsStr = "C:/Users/nadin/OneDrive/Documents/Uni Stuff/Image Processing/PROJECT/dataset/images/Cars"
index = 437
readImage = cv2.imread(pathAsStr+str(index)+'.png')
#show_images([readImage])
extractedPlate, location = preprocessing(readImage)
if location is not None:
    partition = RegionOfInterest(extractedPlate, location)
    ListOfSegChars = segmnetCharactersByContours(partition)
    filename = directory+'done'+str(index)+'.png'
    cv2.imwrite(filename,partition*255)

In [ ]:
def GUImode(toDisplay):
    sg.theme('DarkAmber')   # Add a touch of color
    # All the stuff inside your window.
    layout = [  [sg.Text('Some text on Row 1')],
                [sg.Text('Enter something on Row 2'), sg.InputText()],
                [sg.Button('Ok'), sg.Button('Cancel')] ]

    # Create the Window
    window = sg.Window('Window Title', layout)
    # Event Loop to process "events" and get the "values" of the inputs
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
            break
        print('You entered ', values[0])

    window.close()
    
GUImode(extractedPlate)

## Garbage and another attemted solutions

In [ ]:
#show_images([image, gray_filtered,edged, new_image2, plateRegionOnly ])
    

'''
left_bottom =  location[3][0]
right_bottom = location[0][0]
left_top =  location[2][0]
right_top = location[1][0]
print(right_bottom, left_bottom, left_top, right_top )
xEnd = max(right_top[0],right_bottom[0]) + 20
yEnd = min(right_bottom[1],left_bottom[1]) + 10
plateRegionOnly = extractedPlate[yEnd:left_top[1]+10,left_bottom[0]-20:xEnd ]
print(yEnd, left_top[1],left_bottom[0]-20,xEnd)
print('Plate reg')
show_images([plateRegionOnly])
print('type is -->', type(plateRegionOnly))
'''
#print( x, y, w, h)
    #left_bottom = vertices[3][0]
    #right_bottom = vertices[0][0]
    #left_top =  vertices[2][0]
    #right_top = vertices[1][0]
    
 '''
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 70 and w < 250
        keepHeight = h > 45 and h < 120
        keepArea = area < 6000 and area > 1500
        # ensure the connected component we are examining passes all
        # three tests
        if keepArea:
            # construct a mask for the current connected component and
            # then take the bitwise OR with the mask
            #print("keeping connected component #'{}'".format(i))
            componentMask = (labels == i).astype("uint8") * 255
            mask = cv2.bitwise_or(mask, componentMask)
            show_images([mask])
            mask = np.zeros(threshAdjusted.shape, dtype="uint8")
    '''


    
    '''
    hypotenuse =  math.pow((math.pow((left_bottom[0]-right_bottom[0]), 2) + math.pow((left_bottom[1]-right_bottom[1]), 2)), 0.5)
    opposite = right_bottom[1] - left_bottom[1]
    rotationAngle = math.asin(opposite/hypotenuse)*57.2958
    rotated = imutils.rotate_bound(plate, angle=-rotationAngle)
    '''
    #show_images([plate])
    #show_images([rotated])